In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
import time
import torch
import torchvision
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch import nn, optim
import numpy as np

# Configuration
data_dir = '/content/drive/MyDrive/JPEGImages'
batch_size = 64
num_workers = 8
num_epochs = 10
learning_rate = 0.001
image_size = 224

In [9]:
# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Data loading
def load_data(data_dir, transforms):
    dataset = datasets.ImageFolder(root=data_dir, transform=transforms)
    return dataset

train_dataset = load_data(data_dir, train_transforms)
test_dataset = load_data(data_dir, test_transforms)

# Handling class imbalance
class_count = np.array([len(np.where(train_dataset.targets == i)[0]) for i in range(len(train_dataset.classes))])
class_weights = torch.tensor(np.reciprocal(class_count + 1e-6), dtype=torch.float)
sample_weights = class_weights[train_dataset.targets]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

<ipython-input-9-0ba026c24c3b>:25: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  class_count = np.array([len(np.where(train_dataset.targets == i)[0]) for i in range(len(train_dataset.classes))])


In [10]:
# Model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 50)
)

# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
# Training loop

def train_model(model, train_loader, optimizer, loss_fn, epochs):
    model.train()
    start_time = time.time()  # Start timing
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if (i + 1) % 20 == 0:
                print(f"Epoch {epoch + 1}/{epochs}, Step {i + 1}, Loss: {running_loss / 100:.4f}")
                running_loss = 0.0
    end_time = time.time()  # End timing
    total_time = end_time - start_time
    print(f"Total training time: {total_time:.2f} seconds")


In [13]:
# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')

In [14]:
# Execute training and evaluation
train_model(model, train_loader, optimizer, loss_fn, num_epochs)
evaluate_model(model, test_loader)

Epoch 1/10, Step 20, Loss: 0.5676
Epoch 1/10, Step 40, Loss: 0.4622
Epoch 1/10, Step 60, Loss: 0.4057
Epoch 1/10, Step 80, Loss: 0.3628
Epoch 1/10, Step 100, Loss: 0.3472
Epoch 1/10, Step 120, Loss: 0.3353
Epoch 1/10, Step 140, Loss: 0.3204
Epoch 1/10, Step 160, Loss: 0.3001
Epoch 2/10, Step 20, Loss: 0.2586
Epoch 2/10, Step 40, Loss: 0.2799
Epoch 2/10, Step 60, Loss: 0.2973
Epoch 2/10, Step 80, Loss: 0.2795
Epoch 2/10, Step 100, Loss: 0.2569
Epoch 2/10, Step 120, Loss: 0.2370
Epoch 2/10, Step 140, Loss: 0.2290
Epoch 2/10, Step 160, Loss: 0.2225
Epoch 3/10, Step 20, Loss: 0.2259
Epoch 3/10, Step 40, Loss: 0.2044
Epoch 3/10, Step 60, Loss: 0.2119
Epoch 3/10, Step 80, Loss: 0.2192
Epoch 3/10, Step 100, Loss: 0.2065
Epoch 3/10, Step 120, Loss: 0.2058
Epoch 3/10, Step 140, Loss: 0.2084
Epoch 3/10, Step 160, Loss: 0.1903
Epoch 4/10, Step 20, Loss: 0.1885
Epoch 4/10, Step 40, Loss: 0.1741
Epoch 4/10, Step 60, Loss: 0.1703
Epoch 4/10, Step 80, Loss: 0.1759
Epoch 4/10, Step 100, Loss: 0.1794
E